<a href="https://colab.research.google.com/github/BSNDAN/Airplane_ML/blob/main/merging_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive gcsfs 

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from datetime import datetime 
from datetime import date

In [ ]:
auth.authenticate_user()

# Import tables

In [ ]:
Customer_table = (pd.read_csv(f"gs://hublot-raw-area/files/Data_Extract_V4/Customer_Table.csv",
                            dtype={'Email' : str},)
                  .rename({"Billing Country (text only)" : "Billing_Country", 
                            "Mailing State/Province (text only)" : "Mailing_State_Province",
                            "Birthdate" :'BIRTHDAY',
                            "Person Account: Mailing Zip/Postal Code" : "Mailing_Zip_PostalCode",
                            "Mailing Country (text only)" : "Mailing_Country"}, axis=1))  
  
#Delete space in columns name       
Customer_table.columns = Customer_table.columns.str.replace(" ","_")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
Transactions_table = pd.read_csv(f"gs://hublot-raw-area/files/Data_Extract/Transaction_Table.csv",)
Transactions_table.columns = Transactions_table.columns.str.replace(" ","_")

In [ ]:
Product_table = (pd.read_csv(f"gs://hublot-raw-area/files/Data_Extract_V4/Product_Table.csv")
                .rename(columns={"Product Code" : "Product Reference"}))
Product_table.columns = Product_table.columns.str.replace(" ","_")

In [ ]:
Product_table.shape

(12489, 7)

In [ ]:
Boutique_table = pd.read_csv(f"gs://hublot-raw-area/files/Data_Extract/Boutique_Table.csv")
Boutique_table.columns = Boutique_table.columns.str.replace(" ","_")

In [ ]:
Belonging_table = pd.read_csv(f"gs://hublot-raw-area/files/Data_Extract/Belonging_Table.csv")
Belonging_table.columns = Belonging_table.columns.str.replace(" ","_")

In [ ]:
Customer_table.shape

(175429, 22)

# Merging Transactions and Boutique tables
- merge transactions and boutique on left
- For each transaction, get boutique informations corresponding

In [ ]:
Transactions_boutique_merged = pd.merge(left=Transactions_table, right=Boutique_table, how="left", 
                                        on="Boutique_ID", suffixes=["_transaction","_boutique"])

# Merging Transactions and product tables
- merge transactions and product on left
- For each transaction, get product informations corresponding

In [ ]:
Transactions_df = pd.merge(left=Transactions_boutique_merged, right=Product_table, how="left", 
                                        on="Product_Reference")

# Merging customer and belonging tables
- merge Belongings and Customers on left with customer id
- For each Belonging, get customer informations corresponding
- 36.72% of customer lost

In [ ]:
Customer_table["is_client"] = 1

In [ ]:
Belonging_df = pd.merge(left=Belonging_table, right=Customer_table, how="left", 
                                        on="Customer_ID")

In [ ]:
nb_customer_both = len(set(list(Belonging_df.Customer_ID.unique())) & 
                       set(list(Customer_table.Customer_ID.unique())))

nb_customer_not_in_Belonging = len(Customer_table.Customer_ID) - nb_customer_both
pct_customer_not_in_Belonging = nb_customer_not_in_Belonging/len(Customer_table.Customer_ID)*100

print(f"Number of Customer lost : {nb_customer_not_in_Belonging}")
print(f"Percentage of Customer lost : {np.round(pct_customer_not_in_Belonging, 2)}%")

Number of Customer lost : 64426
Percentage of Customer lost : 36.72%


In [ ]:
Belonging_df.is_client.isna().sum()

25

# Merging Belonging and Transactions tables (data check)

- Merge Belonging and transaction on left with identical columns 

In [ ]:
identical_columns = list(set(list(Transactions_table.columns)) & set(list(Belonging_table.columns)))
identical_columns

['Boutique_ID', 'Product_Reference', 'Customer_ID', 'Serial_Number']

In [ ]:
Belonging_extension = pd.merge(left=Belonging_df, right=Transactions_table, how="left", on=identical_columns)